In [16]:
import csv
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from numpy import nan
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE

from sklearn.utils import resample
plt.style.use('classic')
%matplotlib inline
pd.set_option('display.max_rows', None)

In [17]:
df_train1 = pd.read_csv('sample_data/X_trainData_1.csv')
df_train2 = pd.read_csv('sample_data/Y_trainData_1.csv')
df_test = pd.read_csv('sample_data/X_testData_1.csv')
df_test = df_test.loc[:, df_test.columns != 'ID']

In [18]:
#output = df_train2.drop_duplicates()
#output.groupby('Unnamed: 0').size()
print(df_train1.shape)
print(df_train2.shape)
print(df_test.shape)

(3393, 222)
(3393, 11)
(1000, 222)


In [94]:
res = pd.concat([df_train1, df_train2], axis=1)
res = res[res.OFLX!= -1]
res2=res[res.OFLX==1]
# res3=res[res.OFLX==0]
# df_upsampled = resample(res3,replace=True, n_samples=203,random_state=123)   
# res=pd.concat([res2, df_upsampled], axis=0)
y = res['OFLX']
X = res.drop(['OFLX'], axis = 1) 
X = X.drop(['INH'], axis = 1) 
X = X.drop(['RIF'], axis = 1) 
X = X.drop(['CIP'], axis = 1) 
X = X.drop(['PZA'], axis = 1) 
X = X.drop(['EMB'], axis = 1)
X = X.drop(['CAP'], axis = 1)
X = X.drop(['STR'], axis = 1)
X = X.drop(['AMK'], axis = 1)
X = X.drop(['MOXI'], axis = 1)  
X = X.drop(['KAN'], axis = 1)  

print(y.shape)
print(X.shape)
print(res.shape)
print(res2.shape)
# print(y)
# print(X)
# upsamp = SMOTE(ratio='minority')
# X_sm, y_sm = upsamp.fit_sample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.1)


(690,)
(690, 222)
(690, 233)
(603, 233)


In [172]:
#Best model
model = RandomForestClassifier(n_estimators=567, criterion='entropy', max_depth=208, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split=2, bootstrap=False, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(df_test)
df = pd.read_csv('sample_data/Y_testData_1_nolabels_OFLX.csv')
ID = df['ID'].tolist()
print(y_pred.shape)

(1000, 2)


In [182]:
li_id = []
li_val = []
for i in ID:
    # print(i, y_pred[i-1])
    li_id.append(i)
    li_val.append((y_pred[i-1][1]))
csv = pd.DataFrame({'ID': li_id, 'OFLX' :li_val})
csv.to_csv('OFLX-submit.csv', index=False, header=True)  
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))  
print(roc_auc_score(y_test, y_pred))

[[ 6  6]
 [ 0 57]]
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        12
           1       0.90      1.00      0.95        57

    accuracy                           0.91        69
   macro avg       0.95      0.75      0.81        69
weighted avg       0.92      0.91      0.90        69

0.75
